In [1]:
#Basics 
import numpy as np
import pandas as pd 
import math, random
import sys, os 
import sklearn
from sklearn.metrics import balanced_accuracy_score, RocCurveDisplay
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import log_loss
from scipy import stats 
import scipy
import matplotlib 
import matplotlib.pyplot as plt 
from glob import glob
from itertools import cycle
from joblib import dump, load
%matplotlib inline

#Google Drive setup 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

#Navigate to directory with Google Drive
%cd /content/drive/Shareddrives/CS\ 272\ PCOS\ FL/Model\ Evaluation/

/content/drive/Shareddrives/CS 272 PCOS FL/Model Evaluation


In [3]:
fedavg = None
fedavgm = None
fedprox = None

with np.load('best_fl_models/MLP/fedprox1_1000.npz') as data:
  # for key in data.files:
  #   print(key)
  fedprox = MLPClassifier(hidden_layer_sizes=(64,64), warm_start=True).partial_fit(np.random.uniform(size=(10,13)), np.arange(10), classes=np.arange(10))
  fedprox.coefs_[0] = data['arr_0']
  fedprox.intercepts_[0] = data['arr_1']
  fedprox.coefs_[1] = data['arr_2']
  fedprox.intercepts_[1] = data['arr_3']
  fedprox.coefs_[2] = data['arr_4']
  fedprox.intercepts_[2] = data['arr_5']

with np.load('best_fl_models/MLP/fedavgm3_850.npz') as data:
  # for key in data.files:
  #   print(key)
  fedavgm = MLPClassifier(hidden_layer_sizes=(64,64), warm_start=True).partial_fit(np.random.uniform(size=(10,13)), np.arange(10), classes=np.arange(10))
  fedavgm.coefs_[0] = data['arr_0']
  fedavgm.intercepts_[0] = data['arr_1']
  fedavgm.coefs_[1] = data['arr_2']
  fedavgm.intercepts_[1] = data['arr_3']
  fedavgm.coefs_[2] = data['arr_4']
  fedavgm.intercepts_[2] = data['arr_5']

with np.load('best_fl_models/MLP/fedavg3_550.npz') as data:
  # for key in data.files:
  #   print(key)
  fedavg = MLPClassifier(hidden_layer_sizes=(64,64), warm_start=True).partial_fit(np.random.uniform(size=(10,13)), np.arange(10), classes=np.arange(10))
  fedavg.coefs_[0] = data['arr_0']
  fedavg.intercepts_[0] = data['arr_1']
  fedavg.coefs_[1] = data['arr_2']
  fedavg.intercepts_[1] = data['arr_3']
  fedavg.coefs_[2] = data['arr_4']
  fedavg.intercepts_[2] = data['arr_5']

baseline = load('best_fl_models/mlp_aggregate_baseline.joblib')

In [4]:
#preprocess real test clinic data 
for i in range(10):
  clinic = pd.read_csv(f'../Synthetic Data Generation /clinic_datasets_OOD_challenge_testing/clinic_{i}.csv')
  clinic['response_type'] = clinic.response_type.values - 1
  clinic['treatment'] = clinic.treatment.values - 1
  test = clinic.drop(columns=['response_type', 'treatment'])#.to_numpy()
  for model, model_name in zip((fedprox, fedavgm, fedavg, baseline), ('FedProx', 'FedAvgM', 'FedAvg', 'Baseline Aggregate Model')):
    if model_name=='Baseline Aggregate Model':
      preds = model.predict_proba(test)
    else:
      preds = model.predict_proba(test.to_numpy())
    clinic['pred_treatment'] = np.argmax(preds, axis=1)
    clinic[[f'{j}_prob' for j in range(10)]] = preds
    clinic.to_csv(f'OOD_challenge_test_clinics/MLP/{model_name}/test_clinic_{i}.csv', index=False)
  # fedprox_preds = fedprox.predict_proba(test)
  # fedavgm_preds = fedavgm.predict_proba(test)
  # fedavg_preds = fedavg.predict_proba(test)

In [5]:
# !sudo rm true_test_clinics/baseline_aggr_logreg/*.npy

In [6]:
# with np.load('best_fl_models/test_mlp.npz') as data:
#   for key in data.files:
#     print(key, data[key].shape)
#   fedavg_coefs0 = data['arr_0']
#   fedavg_int0 = data['arr_1']
#   fedavg_coefs1 = data['arr_2']
#   fedavg_int1 = data['arr_3']
#   fedavg_coefs2 = data['arr_4']
#   fedavg_int2 = data['arr_5']

In [7]:

# fedavg.coefs_[0] = fedavg_coefs0
# fedavg.coefs_[1] = fedavg_coefs1
# fedavg.coefs_[2] = fedavg_coefs2
# fedavg.intercepts_[0] = fedavg_int0
# fedavg.intercepts_[1] = fedavg_int1
# fedavg.intercepts_[2] = fedavg_int2